# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Trần Minh Hải Uyên
- MSSV: 21127202
- Lớp: 21CLC04

# Import

In [688]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from IPython.display import display,Math
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [689]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần
val_nsplit = 5
kf = KFold(n_splits = val_nsplit,shuffle = True,random_state=21127202) #Xáo trộn dữ liệu 1 lần duy nhất và trên toàn bộ đặc trưng
model = LinearRegression()

# Cài đặt hàm

In [690]:
def formula_to_latex(formula, features):
    latex_string = f'$$\\text{{Salary}} = {formula[0]:.3f}'
    for i, feature in enumerate(features):
        if formula[i+1] >= 0:
            latex_string += f' + {formula[i+1]:.3f} \\times {feature}'
        else:
            latex_string += f' - {-formula[i+1]:.3f} \\times {feature}'
    latex_string += '$$'
    return latex_string

def kfold_best_feature(features,Ssort = 0,Sprint = 1):
    best_score = float('inf')
    x_train = X_train[features]
    scores = []
    for feature in features:
        x_train_feature = x_train[[feature]]
        for train_index, test_index in kf.split(x_train_feature):
            X_train_kf, X_test_kf = x_train_feature.iloc[train_index], x_train_feature.iloc[test_index]
            y_train_kf, y_test_kf = y_train.iloc[train_index], y_train.iloc[test_index]
            model.fit(X_train_kf, y_train_kf)
            y_pred = model.predict(X_test_kf)
            score = mean_absolute_error(y_test_kf, y_pred)
            scores.append(score)
    # In ra các kết quả cross-validation như yêu cầu
    mae = []
    for i in range(len(features)):
            mae.append((features[i],round(scores[i],3)))
    if (Ssort == 1):
        mae.sort(key=lambda x: x[1])
    for i in range(len(features)):
        if mae[i][1] < best_score:
            best_score = mae[i][1]
            best_feature = mae[i][0]
    if (Sprint == 1):
        print('STT  ', 'Mô hình đặc trưng        ', 'MAE     ')
        for i in range(len(features)):
            print(i+1,'   ',mae[i][0],(24-len(mae[i][0]))*' ',mae[i][1])
        print('Đặc trưng tốt nhất: ', best_feature)
    return best_feature

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [691]:
# Phần code cho yêu cầu 1a
# Chọn các đặc trưng được yêu cầu
features = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain']
x_train = X_train[features]
x_test = X_test[features]

# Huấn luyện mô hình hồi quy tuyến tính
model.fit(x_train, y_train)

# Thể hiện công thức cho mô hình hồi quy
formula = []
formula.append(round(model.intercept_,3))
for i in model.coef_:
    formula.append(round(i,3))
    

In [692]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
# Dự đoán mức lương cho tập kiểm tra và báo cáo kết quả
y_pred = model.predict(x_test)
mae = round(mean_absolute_error(y_test, y_pred),3)
mae

105052.53

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

In [693]:
display(Math(formula_to_latex(formula, features)))

<IPython.core.display.Math object>

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [694]:
# Phần code cho yêu cầu 1b
features = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']
x_train = X_train[features]
x_test = X_test[features]
best_feature = None
# Tìm ra đặc trưng tốt nhất
best_feature = kfold_best_feature(features)

STT   Mô hình đặc trưng         MAE     
1     conscientiousness         123510.521
2     agreeableness             124850.912
3     extraversion              116359.735
4     nueroticism               124353.158
5     openess_to_experience     132187.799
Đặc trưng tốt nhất:  extraversion


In [695]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_personality_feature_model = LinearRegression()
x_train_best_feature = x_train[[best_feature]]
best_personality_feature_model.fit(x_train_best_feature, y_train)

# Thể hiện công thức cho mô hình hồi quy
formula = []
formula.append(round(best_personality_feature_model.intercept_,3))
formula.append(round(best_personality_feature_model.coef_[0],3))
features = []
features.append(best_feature)


In [696]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
x_test_best_feature = x_test[[best_feature]]
y_pred = best_personality_feature_model.predict(x_test_best_feature)
mae = round(mean_absolute_error(y_test, y_pred),3)
mae

118918.709

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)


In [697]:
display(Math(formula_to_latex(formula, features)))

<IPython.core.display.Math object>

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [698]:
# Phần code cho yêu cầu 1c
features = ['English', 'Logical', 'Quant']
best_feature = None
x_train = X_train[features]
x_test = X_test[features]
# Tìm ra đặc trưng tốt nhất
best_feature = kfold_best_feature(features)

STT   Mô hình đặc trưng         MAE     
1     English                   121477.087
2     Logical                   120072.695
3     Quant                     114351.776
Đặc trưng tốt nhất:  Quant


In [699]:

# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_skill_feature_model = LinearRegression()
x_train_best_feature = x_train[[best_feature]]
best_skill_feature_model.fit(x_train_best_feature, y_train)

# Thể hiện công thức cho mô hình hồi quy
formula = []
formula.append(round(best_skill_feature_model.intercept_,3))
formula.append(round(best_skill_feature_model.coef_[0],3))
features = []
features.append(best_feature)

In [700]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
x_test_best_feature = x_test[[best_feature]]
y_pred = best_skill_feature_model.predict(x_test_best_feature)
mae = round(mean_absolute_error(y_test, y_pred),3)
mae

108814.06

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)


In [701]:
display(Math(formula_to_latex(formula, features)))

<IPython.core.display.Math object>

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

- Tìm và so sánh các giá trị MAE trong 23 đặc trưng giải thích

In [702]:
kfold_best_feature(['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain', 'ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg', 'conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience'],1)

STT   Mô hình đặc trưng         MAE     
1     English                   111614.184
2     ElectronicsAndSemicon     111980.392
3     openess_to_experience     115880.773
4     12percentage              116430.138
5     Domain                    117528.0
6     collegeGPA                117579.273
7     Logical                   117794.226
8     CivilEngg                 118745.751
9     conscientiousness         120499.178
10     ComputerScience           121085.707
11     CollegeCityTier           121128.738
12     extraversion              122166.771
13     CollegeTier               122214.594
14     Gender                    122363.205
15     ComputerProgramming       122508.962
16     TelecomEngg               122544.739
17     ElectricalEngg            123067.867
18     nueroticism               124956.765
19     10percentage              124958.262
20     Quant                     126563.519
21     MechanicalEngg            126914.259
22     agreeableness             131757.321
23

'English'

### Mô hình 01 
Từ kết quả trên, chọn ra các đặc trưng có MAE < `120000`. Ta có mô hình 01 gồm các thuộc tính sau: `English`, `ElectronicsAndSemicon`, `openess_to_experience`, `12percentage`, `Domain`, `collegeGPA`, `Logical`, `CivilEngg`

In [703]:
def Module01():
    features = []
    features = ['English', 'ElectronicsAndSemicon', 'openess_to_experience', '12percentage', 
                'Domain', 'collegeGPA', 'Logical', 'CivilEngg']
    return features

### Mô hình 02
Dựa trên các điểm số trong bài thi AMCAT, ta chia thành 3 nhóm như sau:
- Nhóm phần thi khả năng gồm các đặc trưng `English`, `Logical`, `Quant`
- Nhóm phần thi về lập trình, máy tính và kỹ thuật gồm các đặc trưng `Domain`, `ComputerProgramming`, `ElectronicsAndSemicon`, `ComputerScience`, `MechanicalEngg`, `ElectricalEngg`, `TelecomEngg`, `CivilEngg`
- Nhóm phần kiểm tra tính cách `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`

Từ mỗi nhóm, ta lấy một đặc trưng tốt nhất để tạo thành mô hình 02

In [704]:
def Module02():
    features = []
    features.append(kfold_best_feature(['English', 'Logical', 'Quant'],0,0))
    features.append(kfold_best_feature(['Domain', 'ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience',
                                        'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg'],0,0))
    features.append(kfold_best_feature(['conscientiousness', 'agreeableness', 'extraversion',
                                        'nueroticism', 'openess_to_experience'],0,0))
    return features

### Mô hình 03
Kết hợp và chia nhóm 23 đặc trưng, ta có được các nhóm đặc trưng sau:
- Nhóm đặc trưng giới tính gồm đặc trưng `Gender`
- Nhóm đặc trưng quá trình trước Đại học gồm các đặc trưng `10percentage`, `12percentage`
- Nhóm đặc trưng quá trình Đại học gồm các đặc trưng `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`
- 3 nhóm đặc trưng từ bài thi AMCAT giống với mô hình 02

Từ mỗi nhóm, ta lấy một đặc trưng tốt nhất để tạo thành mô hình 03

In [705]:
def Module03():
    features = []
    features.append('Gender')
    features.append(kfold_best_feature(['10percentage', '12percentage'],0,0))
    features.append(kfold_best_feature(['CollegeTier', 'Degree','collegeGPA', 'CollegeCityTier'],0,0))
    features.append(kfold_best_feature(['English', 'Logical', 'Quant'],0,0))
    features.append(kfold_best_feature(['Domain', 'ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience', 
                                        'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg'],0,0))
    features.append(kfold_best_feature(['conscientiousness', 'agreeableness', 'extraversion', 
                                        'nueroticism', 'openess_to_experience'],0,0))
    return features

## Thử nghiệm, so sánh các mô hình

In [706]:
# Phần code cho yêu cầu 1d
module1 = Module01()
module2 = Module02()
module3 = Module03()
models = [module1, module2, module3]
avg_mae = []
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
for module in models:
    total_mae = 0.0
    for train_index, val_index in kf.split(X_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]

        selected_features = module
        num_features = len(selected_features)
        # Chọn các đặc trưng được định nghĩa bởi mô hình
        X_fold_train_selected = X_fold_train[selected_features]
        X_fold_val_selected = X_fold_val[selected_features]

        # Huấn luyện mô hình Linear Regression
        model = LinearRegression()
        model.fit(X_fold_train_selected, y_fold_train)

        # Đưa ra dự đoán trên tập validation
        y_pred_val = model.predict(X_fold_val_selected)

        # Tính toán mean absolute error
        fold_mae = mean_absolute_error(y_fold_val, y_pred_val)
        total_mae += fold_mae

    average_mae = total_mae / val_nsplit
    avg_mae.append(average_mae)


# In ra các kết quả cross-validation như yêu cầu
count = 0
best_score = float('inf')
for i in models:
    count+= 1
    print('Mô hình thứ ', count, i)
print ('STT     ','Mô hình      ','MAE      ')
for i in range(3):
    print(i+1,'Mô hình ',i+1,'          ',avg_mae[i])
    if avg_mae[i] < best_score:
        best_score = avg_mae[i]
        my_best_feature = models[i]

my_best_feature

Mô hình thứ  1 ['English', 'ElectronicsAndSemicon', 'openess_to_experience', '12percentage', 'Domain', 'collegeGPA', 'Logical', 'CivilEngg']
Mô hình thứ  2 ['Quant', 'CivilEngg', 'extraversion']
Mô hình thứ  3 ['Gender', '10percentage', 'collegeGPA', 'Quant', 'CivilEngg', 'extraversion']
STT      Mô hình       MAE      
1 Mô hình  1            114894.94915798395
2 Mô hình  2            117162.71400368749
3 Mô hình  3            113864.05401289689


['Gender', '10percentage', 'collegeGPA', 'Quant', 'CivilEngg', 'extraversion']

In [707]:
#Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
x_train = X_train[my_best_feature]
x_test = X_test[my_best_feature]

# Huấn luyện mô hình hồi quy tuyến tính
my_best_model = LinearRegression()
my_best_model.fit(x_train, y_train)

# Thể hiện công thức cho mô hình hồi quy
formula = []
formula.append(round(my_best_model.intercept_,3))
for i in model.coef_:
    formula.append(round(i,3))

In [708]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
y_pred = my_best_model.predict(x_test)
mae = round(mean_absolute_error(y_test, y_pred),3)
mae

104911.133

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)


In [709]:
display(Math(formula_to_latex(formula, my_best_feature)))

<IPython.core.display.Math object>